# 実際のデータを読み込む
## 佐賀県の人口を例に

In [ ]:
import pandas
import re

In [ ]:
#url = 'https://data.bodik.jp/dataset/77e0cc66-c15d-4473-b3df-2664fe8e2e63/resource/8dc71515-526a-4168-866c-05d2cc8dad7b/download/jinkou.xlsx'
url = 'jinkou.xlsx'

## 表中の和暦を西暦に変換する辞書型
表中では、年が和暦であり、さらに元号が省略されている部分がある。そこで、表中の年を西暦へ変更する辞書型を準備しておく。

In [ ]:
wareki ={
        '大正9年':1920, '昭和5年':1930, '10年':1935, '15年':1940,
        '20年':1945, '25年':1950, '30年':1955, '35年':1960,'40年':1965,
        '45年':1970, '50年':1975, '55年':1980, '60年':1985, '平成2年':1990,
        '7年':1995, '12年':2000, '17年':2005, '22年':2010, '27年':2015}

# 行名の置き換え
- A列とB列に行名があるが、二列に分かれている
- V列は行名として使えそう
- V列の行名からA列とB列に行名を使ったより適切な名前を辞書型で生成

In [ ]:
def getIndex(url:str) -> dict[str,str]:
    """
    URLで指定されたExcelで、V列に対応する適切な行名を取得する
    """
    dd  = pandas.read_excel(url, header = 3,  usecols = 'A:B,V',
        index_col = 2, skiprows = [4, 18, 27], skipfooter = 8) 
    newIndex :dict[str,str]= dict()
    for k in dd.index:
        v = dd['項'][k]
        if type(v) != str:
            v = dd['目'][k]
        v = re.sub('\s','',v)
        newIndex[k] = v.strip()
    return newIndex

## Excelファイル読み込み
- C列からV列を読み込む
- 一部の行を無視
- フッタ部分を無視
- 分かりやすい行名に入れ替え

In [ ]:
def readData(url:str) -> pandas.DataFrame:
    """
    urlで指定されたExcelファイルを読む
    """
    data = pandas.read_excel(url, header = 3, usecols = 'C:V',
             index_col = 19, skiprows = [4, 18, 27], skipfooter = 8) 
    keys = getIndex(url)
    return data.rename(columns = wareki, index = keys)

## メイン部分

- CSVファイルを読む。
- ラベルの整理を行う。
- 整理したラベルを印刷する。

In [ ]:
df :pandas.DataFrame = readData(url)
print(df)
newIndex:dict[str,str] = getIndex(url)
print('新しく取得した行のラベル')
for k in newIndex.keys():
    print(f'{k}->{newIndex[k]}')
